# import def func

In [3]:
# import logging
import sys
# stdout=sys.stdout
sys.path.append('../../')
import os
import random
import time

# import matplotlib
# import matplotlib.pyplot as plt
# import mplfinance as mpf
# sys.stdout=stdout
import numpy as np
import pandas as pd
from JohnsonUtil import commonTips as cct
from JohnsonUtil import johnson_cons as ct
from JSONData import sina_data
from JSONData import tdx_data_Day as tdd
from JSONData import tdx_hdf5_api as h5a

# from mplfinance.original_flavor import candlestick_ohlc
# from JSONData import tdx_hdf5_api as h5a

# %matplotlib widget
# plt.rc('font', family='SimHei', size=13)
# matplotlib.get_backend()
# jupyter nbextension enable --py widgetsnbextension

os.environ['NUMEXPR_MAX_THREADS'] = '12'
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"
import nest_asyncio
nest_asyncio.apply()

In [4]:
f'lastpTDX_DF{resample}' in locals()

NameError: name 'resample' is not defined

In [5]:
def show_tdx_data(df, codelist=None, market_value='3', col=None,limit=20):
    col_src = [
        "name", "couts", "date", "perc", "ra_diff","xratio", "idx", "hat", "close",
        "volsum5d", "hmax", "lmin", "perc1d", "ma20", "ma250", "nclose",
        "llow", 'vol1d', 'vol2d', 'vol3d', 'vol4d', 'vol5d', "perc%sd" %
        (market_value),
        "perc5d"]

    if isinstance(df, pd.Series):
        col_name = [co for co in col_src if co in df.index]
        # col_name.insert(1,'date')
    else:
        col_name = [co for co in col_src if co in df.columns]

    # print(col_name)
    # if isinstance(df, pd.DataFrame) and 'name' in df.columns:
    #     columns_ = col_name
    # else:
    #     if isinstance(df, pd.Series) and 'name' in df.index:
    #         columns_ = col_name.insert(1, "date")
    #     else:
    #         columns_ = col_name.remove("name")
    
    if isinstance(col,list):
        col_name.extend(col)
    elif col is not None:
        col_name.insert(4, col)

    if codelist is None:
        if isinstance(df, pd.Series):
            print("Count:1")
            temp = df.loc[col_name][:limit]
        else:
            print("Count:%s %s" % (df.shape))
            temp = df.loc[:, col_name][:limit]
    else:
        temp = df.loc[[x for x in codelist if x in df.index], col_name][:limit]
    return temp

def clean_sina_dm(dd):
    if (cct.get_now_time_int() > 915 and cct.get_now_time_int() < 926):
        dd = dd[dd.close > 0]
    else:
        dd = dd[dd.open > 0]
    return dd

global sina_dm
sina_dm = None


def get_sina_data_cname(df, dm=None, index=False, realtime=False):
    # index_status=False
    global sina_dm
    if dm is None:
        if sina_dm is None:
            dm = sina_data.Sina().market('all')
        else:
            dm = sina_dm.copy()

    if 'ticktime' in df.columns:
        time_l = time.mktime(dm.loc[:, 'ticktime'][-1].timetuple())
        dur = round((time.time() - time_l) / 60, 2)
        if dur > 5:
            print("timel > 5,renew dm")
            dm = sina_data.Sina().market('all')
            sina_dm = dm.copy()

    if realtime:
        print("compute percent")
        df.rename(columns={'low': 'llow',
                  'high': 'lhigh', 'open': 'lopen'}, inplace=True)
        if 'nclose' in dm.columns:
            dd = cct.combine_dataFrame(
                df, dm.loc
                [:,
                 ['nclose', 'close', 'name', 'open', 'high', 'low', 'llastp',
                  'ticktime']],
                clean=True)
        else:
            dd = cct.combine_dataFrame(
                df, dm.loc
                [:, ['close', 'name', 'open', 'high', 'low', 'llastp', 'ticktime']],
                clean=True)
        dd['perc'] = ((dd['close'] - dd['llastp']) / dd['llastp'] *
                      100).map(lambda x: round(x, 2))
        # dd = cct.combine_dataFrame(df, dm['name'])

    else:
        dd = cct.combine_dataFrame(df, dm['name'])
    # cname = sina_data.Sina().get_code_cname(code)
    return dd

# from JSONData import tdx_hdf5_api as h5a
names = locals()
def get_tdx_all_now(resample='d',market_blk='all',showtable=True):
    top_now = tdd.getSinaAlldf(market=market_blk, vol=ct.json_countVol, vtype=ct.json_countType)
    if resample=='d' or resample=='3d':
        dl='60'
    elif  resample=='w':
        dl='200'
    else:
        dl='300'
    filter='y'
#     tdx_df = f'lastpTDX_DF_{resample}'
    if not f'lastpTDX_DF_{resample}' in names:
        print(f'lastpTDX_DF_{resample} not in locals')
        top_all, lastpTDX_DF = tdd.get_append_lastp_to_df(top_now, dl=dl,resample=resample,showtable=showtable)
        names[f'lastpTDX_DF_{resample}'] = lastpTDX_DF
    else:
        print(f'lastpTDX_DF_{resample} in locals')
        top_all = tdd.get_append_lastp_to_df(top_now, eval(f'lastpTDX_DF_{resample}'))
    return top_all

def get_tdx_all_from_h5_old(resample='d',showtable=True):
    #sina_monitor
    h5_fname = 'tdx_last_df'
    resample=resample
    if resample=='d' or resample=='3d':
        dl='60'
    elif  resample=='w':
        dl='200'
    else:
        dl='300'
    filter='y'
    h5_table = 'low' + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
    h5 = h5a.load_hdf_db(h5_fname, table=h5_table,code_l=None, timelimit=False,showtable=showtable)
    if h5 is None:
        h5,tdx = tdd.get_append_lastp_to_df(sina_data.Sina().market('all'),dl=dl,resample=resample)
    return h5


# def get_tdx_all_from_h5_old(resample='d',dl=ct.duration_date_day, end=None, ptype='low', filter='y',showtable=True):
#     if end is not None:
#         h5_table = ptype + '_' + resample + '_' + str(dl) + '_' + filter + \
#             '_' + end.replace('-', '') + '_' + 'all'
#     else:
#         h5_table = ptype + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
#     #sina_monitor
#     h5_fname = 'tdx_last_df'
#     resample=resample
#     if resample=='d' or resample=='3d':
#         dl='60'
#     elif  resample=='w':
#         dl='200'
#     else:
#         dl='300'
#     filter='y'
#     h5_table = 'low' + '_' + resample + '_' + str(dl) + '_' + filter + '_' + 'all'
#     h5 = h5a.load_hdf_db(h5_fname, table=h5_table,code_l=None, timelimit=False,showtable=showtable)
#     return h5

def write_code_to_blk(codew, blk='077',rewrite=False):
    block_path = tdd.get_tdx_dir_blocknew() + '%s.blk' % (blk)
    write_blk = 'n'
    if not rewrite:
        write_blk = cct.cct_raw_input("write blk [Y] or [N]:")
        if write_blk == 'y' or write_blk == 'Y':
            hdf5_wri = cct.cct_raw_input(
                "Rewrite code [Y] or append [N](defalut:N):")
            if hdf5_wri == 'y' or hdf5_wri == 'Y':
                append_status = False
            else:
                append_status = True
            if len(codew) > 3:
                cct.write_to_blocknew(block_path, codew, append=append_status,
                                      doubleFile=False, keep_last=0)
                print("write:%s block_path:%s" % (len(codew), block_path))
            else:
                print("write error:%s block_path:%s" % (len(codew), block_path))
    else:
        if len(codew) > 3:
                cct.write_to_blocknew(block_path, codew, append=False,
                                      doubleFile=False, keep_last=0)
                print("write:%s block_path:%s" % (len(codew), block_path))
        else:
            print("write error:%s block_path:%s" % (len(codew), block_path))

def apply_col_toint(df, col=None):
    if col is None:
        co2int = ['boll', 'op', 'ratio', 'fib', 'fibl', 'df2']
    # co2int.extend([co for co in df.columns.tolist()
    #                if co.startswith('perc') and co.endswith('d')])
        co2int.extend(['top10', 'topR'])
    else:
        co2int = col
    co2int = [inx for inx in co2int if inx in df.columns]

    for co in co2int:
        df[co] = df[co].astype(int)

    return df

def readHdf5(fpath, root=None):
    if not os.path.exists(fpath):
        print(f'fpath is not exists')
        return None
    store = pd.HDFStore(fpath, "r")
    print(store.keys())
    if len(store.keys()) > 0 and root is None and store is not None:
        root = store.keys()[0].replace("/", "")
        df = store[root]
        df = apply_col_toint(df)
    else:
        df = None
    store.close()
    return df

In [6]:
f'lastpTDX_DF_d' in names
f'lastpTDX_DF_d' in locals()
# eval(f'lastpTDX_DF_d')

False

False

## init data

### get_tdx_all

In [7]:
# h5_day=get_tdx_all_now(resample='d')

### sina

### sina_all

In [8]:
sina_dm = sina_data.Sina().market('all')
lastday = cct.last_tddate()
lastday,sina_dm.name[:1]
sina_dm = clean_sina_dm(sina_dm)

('2025-06-19',
 code
 000533    顺钠股份
 Name: name, dtype: object)

In [9]:
cct.get_time_to_date(time.time(), format='%Y-%D %H:%M:%S')

# abc=dm.loc[:,'ticktime'][-1]
sina_dm.ticktime[-2:]
# dm.ticktime[:1]
# sina_dm.loc[:, 'ticktime'][-1]
# time.strptime(abc,format='%Y-%m-%d %H:%M:%S')
# (time.mktime(abc.timetuple()))
(time.time() - time.mktime(sina_dm.loc[:, 'ticktime'][-1].timetuple())) / 60

'2025-06/20/25 22:41:41'

code
300340   2025-06-20 15:00:00
003019   2025-06-20 15:00:00
Name: ticktime, dtype: datetime64[ns]

461.68760435183844

#### 实时成交

In [10]:
# tdx_path = "G:\\tdx_last_df.h5"
# from JohnsonUtil import commonTips as cct
sina_MultiD_path = f"{cct.get_ramdisk_dir()}\\sina_MultiIndex_data.h5".replace('\\',os.sep)
print(f'sina_MultiD_path:{sina_MultiD_path}')
h5 = readHdf5(sina_MultiD_path)
print(sina_MultiD_path)
if h5 is not None:
    h5.shape
    h5code='603019'
    h5.loc[h5code][-3:]
    code_muti = '603019'
    # h5.loc[code_muti][:2]

    freq = 'D'
    # freq = '5M'
    startime = '09:25:00'
    endtime = '15:01:00'

    time_ratio = cct.get_work_time_ratio()
    time_ratio
    run_col = ['close', 'volume']
    # run_col = ['close']
    mdf = cct.get_limit_multiIndex_freq(
        h5, freq=freq.upper(),
        col=run_col, start=startime, end=endtime, code=None)
    allvol = mdf.volume.sum()
    allvol = round(allvol / 10 / 1000 / 1000,2)
    print("总量:%s 亿"%(allvol))
    mdf['vol']=mdf['volume'].apply(lambda x: round(x /1000/1000,2)) 
    mdf['amount']=(mdf.volume * mdf.close).map(lambda x: round(x/100/1000/1000, 2))
    #全局占比
    mdf['volume'] = mdf.amount.apply(
        lambda x: round(x / allvol * 100 / time_ratio, 2))
    mdf=mdf.reset_index().set_index('code')
    mdf.loc[h5code]
    str(h5.index.get_level_values(1)[-1])[:10]
else:
    mdf = None

sina_MultiD_path:/Volumes/RamDisk/sina_MultiIndex_data.h5
['/all_30']
/Volumes/RamDisk/sina_MultiIndex_data.h5


(2767032, 6)

,close,high,low,llastp,volume,lastbuy
ticktime,,,,,,
2025-06-20 14:58:58,66.9,67.88,66.75,67.55,16223684,67.11
2025-06-20 14:59:40,66.9,67.88,66.75,67.55,16223684,67.18
2025-06-20 15:00:01,66.9,67.88,66.75,67.55,16398684,67.07


1.0

总量:8715.97 亿


ticktime    2025-06-21 00:00:00
close                  67.21773
volume                     0.13
vol                        16.4
amount                    11.02
Name: 603019, dtype: object

'2025-06-20'

#### 读取实时成交个股

In [11]:
if mdf is not None:
    nowdf = mdf.sort_values(by=['volume'], ascending=[0])
    # nowdf.shape, df.shape, code_idx.shape
    # top20_nd = nowdf.reset_index().loc[:, ['code', 'volume']][:20]
    top20_nd = nowdf.reset_index().loc[:, ['code', 'volume']][:20]
    top20_nd = get_sina_data_cname(top20_nd)
    nowdf.shape, top20_nd

((5420, 5),
         volume   name
 code                 
 300308    0.73   中际旭创
 300468    0.71   四方精创
 600519    0.57   贵州茅台
 300476    0.55   胜宏科技
 300502    0.54    新易盛
 601318    0.53   中国平安
 300773    0.49    拉卡拉
 002475    0.45   立讯精密
 002463    0.45   沪电股份
 002104    0.43   恒宝股份
 000858    0.41  五 粮 液
 300059    0.39   东方财富
 300750    0.38   宁德时代
 003040    0.36    楚天龙
 002384    0.35   东山精密
 002017    0.33   东信和平
 603300    0.32   海南华铁
 300255    0.31   常山药业
 002050    0.31   三花智控
 002594    0.29    比亚迪)

##### 合并实时成交量及SinaDM

In [12]:
# mdf[:1]
if mdf is not None:
    sinadm = cct.combine_dataFrame(sina_dm, mdf[['vol','amount']])
else:
    sinadm = sina_dm
sinadm['percent']=((sinadm['close']-sinadm['llastp'])/(sinadm['llastp'])*100).round(1)


## resample 多周期

### filterKey

In [36]:
def get_filter(idx=0):
    filter_key=filter_query[filter_keys[idx]]
    print(f'{filter_keys[idx]}: {filter_key}')
    return filter_key

In [37]:
filterIDX=2
# h5_3d=None,h5_d=None,h5_w=None,h5_m=None

### resample 3d

In [38]:
filter_query={'2周期上冲上轨And涨幅新高': 'lasth1d > upper1 and lasth2d >upper2 and high >boll and percent > maxp',
             '3周期新高上轨':'lastp1d > lastp2d > lastp3d and lasth1d > upper1 and lasth2d > upper2 and lasth3d > upper3',
             '3周期新高大于Ma5': 'lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d'}
filter_keys=list(filter_query.keys())

In [39]:
# h5_3d = get_tdx_all_from_h5(resample='3d') if h5_3d is None else h5_3d
h5_3d = get_tdx_all_now(resample='3d')
# tdx_3d = cct.combine_dataFrame(h5_3d,sinadm)

initdx 

[2025-06-20 22:44:27,967] ERROR:tdx_data_Day.py(get_append_lastp_to_df:4379): tdx Out:3 code:['600705', '601028']


:0 b1>:5184 it:0.5 lastpTDX_DF_3d in locals
tdxdataT:2.52
DLTDXE:2.52 

In [40]:
# tdx_now.query('close >  lastp1d  and lastl1d < ma51d and close > ma51d and lastp1d < upper and lastp1d > lastp2d and lasth1d >high4 and boll > 0 and volume < 1')
# tdx_select = tdx_3d.query('close > lastp1d and percent < 3')
# tdx_select = tdx_3d.query('lastp1d > lastp2d > lastp3d and lasth1d > upper1 and lasth2d > upper2 and lasth3d > upper3')
# tdx_select = tdx_3d.query('lastp1d > lastp2d  and lasth2d > lasth3d and (lasth2d > upper2 or lasth3d > upper3) and lasth2d < hmax')
# tdx_select = tdx_3d.query('lastp1d > lastp2d  and lasth2d > lasth3d and lasth3d > lasth4d and lastl1d > lastl2d and lastl2d > lastl3d')
tdx_select = h5_3d.query(f'{get_filter(filterIDX)}')
show_tdx_data(tdx_select,limit=10)

3周期新高大于Ma5: lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d
Count:192 221


,name,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,
600156,华升股份,0,2025-04-08,7.86,7.00,4.05,1.20,7.86,4.42,22.50,14.3
688205,德科立,0,2025-04-08,65.81,64.25,32.22,40.41,67.69,34.44,1.20,1.2
831010,XD凯添燃,0,2025-04-08,15.96,15.64,10.10,34.41,16.45,10.10,0.00,1.3
836260,中寰股份,0,2025-04-08,14.18,14.36,9.60,0.00,14.48,9.60,0.10,1.2
836221,易实精密,0,2025-04-08,22.00,18.16,11.11,43.50,23.58,11.11,0.20,-1.1
688188,柏楚电子,0,2025-04-08,132.00,141.25,109.19,1.20,133.25,109.19,0.00,1.3
605058,澳弘电子,0,2025-04-08,25.36,24.70,17.80,1.20,25.44,18.94,1.30,1.2
688195,腾景科技,0,2025-04-08,42.59,42.09,29.05,1.20,43.03,30.45,1.20,12.3
873169,七丰精工,0,2025-04-08,40.03,39.53,11.18,31.09,42.84,11.18,33.81,1.3


In [41]:
# t3d_code = tdx_3d.query('lasth1d > lasth2d > lasth3d and close >= high4')
t3d_code = h5_3d.query(f'{get_filter(1)}')
show_tdx_data(t3d_code)

3周期新高上轨: lastp1d > lastp2d > lastp3d and lasth1d > upper1 and lasth2d > upper2 and lasth3d > upper3
Count:80 221


,name,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,
600156,华升股份,0,2025-04-08,7.86,7.00,4.05,1.20,7.86,4.42,22.50,14.30
839719,宁新新材,0,2025-04-08,23.70,15.92,8.90,45.19,22.64,8.90,1.20,-0.90
870204,沪江材料,0,2025-04-08,24.45,22.30,14.09,1.20,25.36,14.09,45.09,0.00
688205,德科立,0,2025-04-08,65.81,64.25,32.22,40.41,67.69,34.44,1.20,1.20
430090,同辉信息,0,2025-04-08,7.77,7.90,4.69,17.70,7.71,4.69,-0.90,14.10
688668,鼎通科技,0,2025-04-08,64.56,58.28,32.09,1.30,65.00,34.25,15.60,16.70
837821,则成电子,0,2025-04-08,32.81,28.47,18.30,46.69,34.22,18.31,1.20,16.80
688498,源杰科技,0,2025-04-08,166.75,147.25,88.12,36.31,165.00,90.62,1.20,12.30
833171,国航远洋,0,2025-04-08,13.62,13.02,5.38,1.20,13.62,5.38,1.20,-0.90


### resample W

In [42]:
# h5_w = get_tdx_all_from_h5(resample='w') if h5_w is None else h5_w

h5_w = get_tdx_all_now(resample='w') 
# tdx_w = cct.combine_dataFrame(h5_w,sinadm)

initdx :0 b1>:5184 it:0.4 lastpTDX_DF_w in locals


In [43]:
# tw_code = tdx_w.query('lasth1d > lasth2d > lasth3d and close >= high4')
# tw_code = tdx_w.query('lasth1d > lasth2d and lasth2d >lasth3d and lasth2d > upper1')
# tw_code = tdx_w.query ('lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d')
# tw_code = tdx_w.query(f'{get_filter(0)}')
# tw_code = tdx_w.query(f'{get_filter(filterIDX)}')
tw_code = h5_w.query(f'{get_filter(filterIDX)}')
show_tdx_data(tw_code,limit=10)

3周期新高大于Ma5: lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d
Count:650 223


,name,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,
301085,亚康股份,0,2025-04-18,55.25,86.06,47.00,1.2,55.69,53.25,0.00,-1.1
688213,思特威,0,2024-11-29,93.12,106.12,68.81,0.0,93.06,68.81,1.30,0.0
301093,华兰股份,0,2025-01-10,31.11,30.28,21.52,0.1,31.09,22.52,0.10,-0.9
002992,宝明科技,0,2025-01-10,56.91,76.06,51.19,0.0,58.34,52.34,1.30,-1.1
000049,德赛电池,0,2025-04-11,21.58,24.81,17.91,0.0,21.72,17.91,0.00,0.0
300890,翔丰华,0,2025-04-30,29.36,34.00,22.25,0.0,29.52,24.09,1.10,-0.1
301045,天禄科技,0,2025-04-11,21.88,24.12,16.50,1.2,22.02,16.50,1.30,-0.9
300615,欣天科技,0,2025-04-11,12.27,16.50,9.00,0.0,12.38,9.00,19.09,-1.1
688618,三旺通信,0,2025-01-10,21.88,24.77,17.19,0.0,21.97,18.69,1.10,-0.1


### resample M

In [44]:
# h5_m = get_tdx_all_from_h5(resample='m') if h5_m is None else h5_m
h5_m = get_tdx_all_now(resample='m')
# tdx_m = cct.combine_dataFrame(h5_m,sinadm)

initdx :0 b1>:5184 it:0.4 lastpTDX_DF_m in locals


In [45]:
# tm_code = tdx_m.query('lastp1d > lastp2d and lasth2d > lasth3d and lastp1d > ma51d')
# tm_code = tdx_m.query('lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d')
# tm_code = tdx_m.query('lasth1d > upper1 and lasth2d >upper2 and high >boll and percent > maxp')
tm_code = h5_m.query(f'{get_filter(filterIDX)}')
show_tdx_data(tm_code)

3周期新高大于Ma5: lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d
Count:946 221


,name,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,
601028,玉龙股份,0,2025-02-28,12.83,14.58,10.52,1.10,0.00,11.14,1.10,0.00
839946,华阳变速,0,2025-01-27,9.42,10.46,2.77,1.10,9.58,6.18,0.00,48.50
001238,浙江正特,0,2024-09-30,37.06,32.69,19.03,1.20,37.19,20.59,0.00,16.30
688321,微芯生物,0,2025-01-27,21.17,23.00,15.15,1.30,21.17,16.52,-1.10,15.30
001268,联合精密,0,2024-12-31,24.12,20.88,14.10,34.09,24.33,16.75,0.00,1.20
003026,中晶科技,0,2025-03-31,31.25,40.97,18.44,1.20,32.03,28.55,0.00,1.20
605289,罗曼股份,0,2025-03-31,28.05,30.05,18.05,1.20,28.12,23.22,0.00,1.20
301022,海泰科,0,2024-12-31,28.19,28.92,16.08,0.00,28.05,20.16,0.00,59.09
301093,华兰股份,0,2024-10-31,31.11,28.08,17.12,1.10,31.09,21.78,0.00,1.20


### resample D

In [46]:
# h5_d = get_tdx_all_from_h5(resample='d') if h5_d is None else h5_d
h5_d = get_tdx_all_now(resample='d') 
# tdx_d = cct.combine_dataFrame(h5_d,sinadm)

initdx 

[2025-06-20 22:44:38,443] ERROR:tdx_data_Day.py(get_append_lastp_to_df:4379): tdx Out:3 code:['600705', '601028']


:0 b1>:5184 it:0.4 lastpTDX_DF_d in locals
tdxdataT:2.58
DLTDXE:2.58 

In [47]:
# td_code = tdx_d.query('lasth1d > lasth2d > lasth3d and close >= high4')
# td_code = tdx_d.query('ma51d > ma10d and ral > 3 and lastp1d >ma20d and low > lastl1d and vol > 2' )
# td_code = tdx_d.query('lastp1d > lastp2d  and lasth1d > lasth2d and lasth2d > lasth3d and lastl1d > lastl2d and lastl2d > lastl3d and close > open')
td_code = h5_d.query(f'{get_filter(filterIDX)}')
show_tdx_data(td_code,limit=10)
if len(td_code) < 30:
    write_code_to_blk(td_code.index,'065',rewrite=True)

3周期新高大于Ma5: lastp1d > lastp2d and lasth2d > lasth3d and lastl2d > lastl3d and lastp1d > ma51d
Count:191 224


,name,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,
688515,裕太微,0,2025-04-07,87.88,104.88,86.75,1.2,88.81,84.00,0.1,-0.10
688100,威胜信息,0,2025-06-13,33.41,35.88,31.52,1.2,33.31,31.52,1.2,-0.10
002850,科达利,0,2025-04-08,108.50,126.12,108.00,1.3,109.38,94.12,0.1,0.00
688630,芯碁微装,0,2025-04-07,77.88,79.06,67.44,1.3,78.50,58.22,-0.1,-0.10
603312,西典新能,0,2025-06-17,34.69,39.44,33.72,1.3,34.84,34.09,0.0,-0.10
603301,振德医疗,0,2025-04-30,20.81,20.28,18.73,0.1,20.84,18.45,1.3,0.00
600519,贵州茅台,0,2025-06-16,1429.00,1635.00,1401.00,0.0,1434.00,1401.00,0.3,-1.10
600156,华升股份,0,2025-04-08,7.86,7.50,5.14,1.2,7.86,4.42,0.0,33.41
300964,本川智能,0,2025-04-07,40.56,40.38,37.12,1.2,40.41,31.73,0.1,-1.10


In [48]:
# td_code2 = tdx_d.query(f'{get_filter(2)}')
td_code2 = h5_d.query('lastp1d > lastp2d  and lasth1d > lasth2d and lasth2d > lasth3d and lastl1d > lastl2d and lastl2d > lastl3d and close > open')
show_tdx_data(td_code2,limit=10)


Count:41 224


,name,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,
600156,华升股份,0,2025-04-08,7.86,7.50,5.14,1.20,7.86,4.42,0.0,33.41
300964,本川智能,0,2025-04-07,40.56,40.38,37.12,1.20,40.41,31.73,0.1,-1.10
839719,宁新新材,0,2025-04-07,23.70,17.45,11.23,1.20,22.64,8.90,0.0,1.20
605389,长龄液压,0,2025-04-08,39.25,35.53,31.20,34.69,39.19,25.70,1.3,1.30
688200,华峰测控,0,2025-06-17,138.12,142.75,132.00,1.30,137.50,128.38,-0.1,0.00
872895,花溪科技,0,2025-04-07,52.53,43.47,18.39,1.20,48.81,13.18,0.3,-1.00
831152,昆工科技,0,2025-04-07,24.44,21.70,18.80,43.09,24.00,15.00,0.1,0.00
688652,京仪装备,0,2025-06-17,54.59,59.09,52.16,1.20,54.47,51.12,-1.1,0.10
833914,远航精密,0,2025-04-07,23.78,20.59,18.08,45.31,23.50,15.07,-1.1,1.20


#### 多周期筛选

In [49]:
# '002476' in td_code

In [50]:
for co in ['tm_code','tw_code','t3d_code','td_code']:
    print(f'{co}:{len(eval(co))}')
code_f=  list(set(tm_code.index) & set(tw_code.index) & set(t3d_code.index)  & set(td_code.index))

if len(code_f) > 0:
    show_tdx_data(tw_code.loc[code_f],limit=10)
else:
    print('no code')

tm_code:946
tw_code:650
t3d_code:80
td_code:191
Count:13 223


,name,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,
872895,花溪科技,0,2025-01-27,52.53,30.20,11.71,1.10,48.81,11.91,1.20,1.20
600156,华升股份,0,2025-01-10,7.86,6.39,4.05,1.20,7.86,4.31,21.70,1.20
839719,宁新新材,0,2025-01-27,23.70,15.48,8.75,55.50,22.64,8.81,0.00,40.59
301357,北方长龙,3,2025-01-27,60.00,39.66,26.45,87.69,62.38,28.48,0.00,1.30
688619,罗普特,1,2025-01-10,12.12,10.25,7.10,1.10,12.36,7.80,1.30,-0.90
300157,新锦动力,164,2025-04-25,5.44,4.01,2.28,66.88,5.65,2.66,4.10,-0.90
002235,安妮股份,269,2025-04-11,9.03,7.29,4.12,34.31,8.88,4.12,1.20,-0.10
002017,东信和平,275,2025-04-11,15.64,11.50,8.07,65.31,15.61,8.07,16.41,-0.10
003040,楚天龙,84,2025-04-11,23.33,16.64,10.23,42.81,21.95,10.23,22.50,-0.10


#### check data

In [52]:
code_check = [co for co in code_f if co in h5_3d.index]
len(code_check)

13

In [53]:
show_tdx_data(td_code.loc[code_f])

Count:13 224


,name,couts,date,close,hmax,lmin,perc1d,nclose,llow,perc3d,perc5d
code,,,,,,,,,,,
872895,花溪科技,0,2025-04-07,52.53,43.47,18.39,1.20,48.81,13.18,0.30,-1.00
600156,华升股份,0,2025-04-08,7.86,7.50,5.14,1.20,7.86,4.42,0.00,33.41
839719,宁新新材,0,2025-04-07,23.70,17.45,11.23,1.20,22.64,8.90,0.00,1.20
301357,北方长龙,3,2025-04-07,60.00,45.00,30.70,62.00,62.38,26.45,0.00,27.30
688619,罗普特,1,2025-04-07,12.12,12.43,9.60,1.10,12.36,7.10,-0.90,1.30
300157,新锦动力,164,2025-04-07,5.44,5.07,3.55,0.00,5.65,2.42,38.91,25.41
002235,安妮股份,269,2025-04-08,9.03,7.12,5.17,1.30,8.88,4.33,0.00,17.30
002017,东信和平,275,2025-04-08,15.64,13.27,9.54,42.09,15.61,8.24,17.30,1.20
003040,楚天龙,84,2025-04-08,23.33,19.20,12.37,47.81,21.95,10.71,0.30,-1.10


### Write toblk

In [54]:
# write_code_to_blk(code_f,'077',rewrite=False)